In [1]:
import os
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_base="https://api.jarvis73.com/v1"
openai.organization = os.getenv("Organization ID")
#os.environ["OPENAI_API_KEY"] = ""

In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from ebm_agent import ebm_agent
from langchain.chat_models import ChatOpenAI
from interpret.glassbox import ExplainableBoostingClassifier
from langchain.memory import ConversationBufferMemory

## 数据训练模型

In [3]:
df = pd.read_csv("./data/spaceship-titanic/train.csv") # replace with path to dataset on your machine
df.head()

dataset_description = ""
y_axis_description = ""

# transform cabin since 8000 unique values do not fit into the context windows of the LLM we want to use
df['Cabin'] = df['Cabin'].map(lambda x: x[:1] + '/' + x[-1] if isinstance(x, str) else x)

# pandas to numpy array
X_data = df.drop(columns=["PassengerId", "Transported", "Name"]).values
y_data = df["Transported"].values

X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=42)

In [4]:
dataset_description = """\nHi there, here is a general description of the data set on which I trained the model. This description is from kaggle:

Welcome to the year 2912, where your data science skills are needed to solve a cosmic mystery. We've received a transmission from four lightyears away and things aren't looking good.

The Spaceship Titanic was an interstellar passenger liner launched a month ago. With almost 13,000 passengers on board, the vessel set out on its maiden voyage transporting emigrants from our solar system to three newly habitable exoplanets orbiting nearby stars.

While rounding Alpha Centauri en route to its first destination—the torrid 55 Cancri E—the unwary Spaceship Titanic collided with a spacetime anomaly hidden within a dust cloud. Sadly, it met a similar fate as its namesake from 1000 years before. Though the ship stayed intact, almost half of the passengers were transported to an alternate dimension!

To help rescue crews and retrieve the lost passengers, you are challenged to predict which passengers were transported by the anomaly using records recovered from the spaceship’s damaged computer system.

The task is to predict whether a passenger was transported to an alternate dimension during the Spaceship Titanic's collision with the spacetime anomaly. To help you make these predictions, you're given a set of personal records recovered from the ship's damaged computer system.

Feature Descriptions:

PassengerId - A unique Id for each passenger. Each Id takes the form gggg_pp where gggg indicates a group the passenger is travelling with and pp is their number within the group. People in a group are often family members, but not always.
HomePlanet - The planet the passenger departed from, typically their planet of permanent residence.
CryoSleep - Indicates whether the passenger elected to be put into suspended animation for the duration of the voyage. Passengers in cryosleep are confined to their cabins.
Cabin - The cabin number where the passenger is staying. Takes the form deck/num/side, where side can be either P for Port or S for Starboard.
Destination - The planet the passenger will be debarking to.
Age - The age of the passenger.
VIP - Whether the passenger has paid for special VIP service during the voyage.
RoomService, FoodCourt, ShoppingMall, Spa, VRDeck - Amount the passenger has billed at each of the Spaceship Titanic's many luxury amenities.
Name - The first and last names of the passenger.
"""

y_axis_description = "The y-axis depicts contributions in log-odds towards the outcome, that is the probability that the passenger was transported to another dimension."

In [5]:
feature_names = df.drop(columns=["PassengerId", "Transported", "Name"]).columns.tolist()

ebm = ExplainableBoostingClassifier(interactions=0, 
                                    feature_names=feature_names)

ebm.fit(X_train, y_train)
ebm.score(X_test, y_test)

D:\anaconda3\lib\site-packages\interpret\glassbox\_ebm\_ebm.py:669: UserWarning: Missing values detected. Our visualizations do not currently display missing values. To retain the glassbox nature of the model you need to either set the missing values to an extreme value like -1000 that will be visible on the graphs, or manually examine the missing value score in ebm.term_scores_[term_index][0]
  warn(


0.78953421506613

In [6]:
llm = ChatOpenAI(model_name= "gpt-3.5-turbo-16k",temperature=0)
llm_gpt4 = ChatOpenAI(model_name= "gpt-4",temperature=0)

In [7]:
df = pd.read_csv("./data/spaceship-titanic/test.csv")

In [8]:
eag = ebm_agent(llm_gpt4,ebm,df = df,dataset_description = dataset_description,y_axis_description = y_axis_description)

INFO: The graph of feature RoomService was simplified by 1.1%.
INFO: The graph of feature FoodCourt was simplified by 0.4%.
INFO: The graph of feature ShoppingMall was simplified by 0.4%.
INFO: The graph of feature Spa was simplified by 0.6%.
INFO: The graph of feature VRDeck was simplified by 0.6%.
INFO: The graph of feature RoomService was simplified by 1.1%.
INFO: The graph of feature FoodCourt was simplified by 0.4%.
INFO: The graph of feature ShoppingMall was simplified by 0.4%.
INFO: The graph of feature Spa was simplified by 0.6%.
INFO: The graph of feature VRDeck was simplified by 0.6%.


In [24]:
ans = eag.run("Summarize the entire model, introduce and describe several features that have a greater impact on the prediction results.")
print(ans)

The Generalized Additive Model (GAM) is used to predict whether a passenger was transported to an alternate dimension during the Spaceship Titanic's collision with the spacetime anomaly. The model uses several features from the passenger's personal records, and each feature contributes differently to the prediction. 

The features with the greatest impact on the prediction results, based on their global feature importances, are "CryoSleep", "RoomService", "Spa", and "VRDeck". 

"CryoSleep" indicates whether the passenger was in suspended animation for the duration of the voyage. The model suggests that passengers in CryoSleep are more likely to be transported to an alternate dimension.

"RoomService" represents the amount the passenger has billed for room service. The model suggests that passengers who spent less on room service were more likely to be transported to another dimension, while those who spent more were less likely.

"Spa" represents the amount a passenger has billed at th

In [25]:
ans = eag.run("visualize the importance of each feature.")
print(ans)

D:\anaconda3\lib\site-packages\langchain\tools\python\tool.py:63: LangChainPendingDeprecationWarning: On 2023-10-27 this module will be be deprecated from langchain, and will be available from the langchain-experimental package.This code is already available in langchain-experimental.See https://github.com/langchain-ai/langchain/discussions/11680.
  warn_deprecated(


alt.Chart(...)

The chart shows the importance of each feature in the model. The features "Spa" and "VRDeck" have the highest importance, followed by "CryoSleep" and "RoomService". The feature "VIP" has the lowest importance. This visualization helps to understand the relative importance of each feature in the model.


In [26]:
ans = eag.run("Add a polyline to the previously generated chart to represent the changing trend of the importance of each feature.")
print(ans)

D:\anaconda3\lib\site-packages\langchain\tools\python\tool.py:63: LangChainPendingDeprecationWarning: On 2023-10-27 this module will be be deprecated from langchain, and will be available from the langchain-experimental package.This code is already available in langchain-experimental.See https://github.com/langchain-ai/langchain/discussions/11680.
  warn_deprecated(


alt.LayerChart(...)

The chart has been successfully generated with a polyline representing the changing trend of the importance of each feature. The line starts from the feature with the lowest importance and ends at the feature with the highest importance, showing a clear upward trend. This visualization helps to understand how the importance of each feature changes in the model.


In [27]:
ans = eag.run("Tell me the basic information of the dataframe data")
print(ans)

D:\anaconda3\lib\site-packages\langchain\tools\python\tool.py:63: LangChainPendingDeprecationWarning: On 2023-10-27 this module will be be deprecated from langchain, and will be available from the langchain-experimental package.This code is already available in langchain-experimental.See https://github.com/langchain-ai/langchain/discussions/11680.
  warn_deprecated(


The dataframe contains 11 entries and 2 columns. The columns are "Feature" and "Importance". The "Feature" column contains the names of the features used in the model and the "Importance" column contains the corresponding importance scores. There are no missing values in the dataframe. The data types of the columns are object and float64. The dataframe uses 304.0+ bytes of memory.


In [28]:
ans = eag.run("Use the seventh to ninth rows of data in the dataframe as prediction samples and tell me the prediction results of each row.")
print(ans)

D:\anaconda3\lib\site-packages\langchain\tools\python\tool.py:63: LangChainPendingDeprecationWarning: On 2023-10-27 this module will be be deprecated from langchain, and will be available from the langchain-experimental package.This code is already available in langchain-experimental.See https://github.com/langchain-ai/langchain/discussions/11680.
  warn_deprecated(


NameError: name 'ebm' is not defined

In [22]:
#清空当前agent的memory
memory = ConversationBufferMemory(memory_key="chat_history")
memory.clear()
eag.memory = memory

ERROR! Session/line number was not unique in database. History logging moved to new session 214


In [20]:
#设置不显示agent执行日志，默认显示
eag.verbose = False